In [6]:
import os
import logging
import json
import psycopg2
import pandas as pd

def connect_to_database():
    # Define your database connection parameters
    db_params = {
        "host": "localhost",
        "database": "blockchain",
        "user": "postgres",
        "password": "auth2020"
    }

    try:
        # Connect to the database
        conn = psycopg2.connect(**db_params)
        print("Connected to the database successfully!")
        return conn
    except psycopg2.Error as e:
        print(f"Error connecting to the database: {e}")
        return None

def get_most_recent_transaction_id(conn):
    table_name_2 = "transaction_header_redelegate"
    id =
    try:
        with conn.cursor() as cursor:
            # Query to retrieve the most recent transaction_exec_id
            get_transaction_id_query = """
                SELECT transaction_exec_id
                FROM transaction_header_exec
                ORDER BY transaction_exec_id DESC
                LIMIT 1;
            """
            cursor.execute(get_transaction_id_query)
            result = cursor.fetchone()
            if result:
                return result[0]
            else:
                print("No transaction_exec_id found.")
                return None
    except psycopg2.Error as e:
        print(f"Error retrieving transaction_exec_id: {e}")
        return None

def main():
    block = '5927808'  # Change this number if you want to load a different file
    file_path = f'C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/{block}.json'
    table_name = 'transaction_detail_msg_begin_redelegate'
    try:
        # Check if the file exists
        if os.path.exists(file_path):
            print(f"Found JSON file: {file_path}")

            # Load JSON data from the file
            with open(file_path, 'r') as json_file:
                json_data = json.load(json_file)

                # Connect to the database
                conn = connect_to_database()

                # Confirm the database connection
                if conn:
                    print("Database connection successful!")

                    # Get the most recent transaction_exec_id
                    transaction_exec_id = get_most_recent_transaction_id(conn)

                    if transaction_exec_id:
                        # Extract data from the JSON structure
                        delegator_address = json_data["tx"]["body"]["messages"][0]["delegator_address"]
                        validator_source_address = json_data["tx"]["body"]["messages"][0]["validator_src_address"]
                        validator_target_address = json_data["tx"]["body"]["messages"][0]["validator_dst_address"]
                        amount_value = json_data["tx"]["body"]["messages"][0]["amount"][0]["amount"]
                        denom_value = json_data["tx"]["body"]["messages"][0]["amount"][0]["denom"]

                        # Construct the insert query for the current message
                        msg_insert_query = f"""
                            INSERT INTO {table_name} (transaction_begin_redelegate_id, delegator_address, validator_source_address, validator_target_address, denom, amount)
                            VALUES ('{transaction_exec_id}', '{delegator_address}', '{validator_source_address}', '{validator_target_address}', '{denom_value}', {amount_value});
                        """

                        # Execute the insert query
                        with conn.cursor() as cursor:
                            cursor.execute(msg_insert_query)
                            conn.commit()  # Commit the transaction

                        print("Data inserted successfully!")
                    else:
                        print("No transaction_id found. Data not inserted.")
                else:
                    print("Database connection failed.")
        else:
            print(f"JSON file not found: {file_path}")
    except Exception as e:
        print(f"Error: {e}")

if __name__ == "__main__":
    main()


Found JSON file: C:/Users/lmhmo/Indexer_Project/indexer_project/Decrypted_files/5927808.json
Connected to the database successfully!
Database connection successful!
Error: 0
